In [1]:
import pyvista as pv
import os

In [2]:
MESH_Z_SCALE = 0.01
PATH = '2022_11_30_11_52_49_mesh_geometry.vtk'
OUT_PATH = f'{os.path.splitext(os.path.basename(PATH))[0]}.msh'

## Load source mesh

In [3]:
mesh = pv.read(PATH)
mesh.clear_data()  # clears any field data
mesh

UnstructuredGrid,Information
N Cells,1841
N Points,1916
X Bounds,"2.318e+02, 2.382e+02"
Y Bounds,"3.943e+01, 5.107e+01"
Z Bounds,"-7.000e+01, 0.000e+00"
N Arrays,0


In [4]:
# Extract surface geometry 
surface = mesh.extract_surface()

# Make sure we didn't loose cells
assert mesh.n_cells == surface.n_cells

surface

PolyData (0x15efef5e0)
  N Cells:    1841
  N Points:   993
  N Strips:   0
  X Bounds:   2.318e+02, 2.382e+02
  Y Bounds:   3.943e+01, 5.107e+01
  Z Bounds:   -7.000e+01, 0.000e+00
  N Arrays:   2

### Display source mesh

In [5]:
pl = pv.Plotter()
pl.add_mesh(surface, show_edges=True)
pl.set_scale(zscale=MESH_Z_SCALE)
pl.show()

Widget(value="<iframe src='http://localhost:61908/index.html?ui=P_0x15f04b050_0&reconnect=auto' style='width: …

## Subdivision

Use PyVista's subdivision filter: https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyDataFilters.subdivide.html#subdivide

In [6]:
# Set the number of subdivisions
N_SUB = 3

In [7]:
sub = surface.subdivide(
    nsub=N_SUB,           # Number of subdivisions
    subfilter='linear',   # Subdividing algorithm
    progress_bar=True,    # Monitor progress for larger meshes
)
sub

Subdividing Mesh: 100%|██████████[00:00<00:00]


PolyData (0x161a99c60)
  N Cells:    117824
  N Points:   59485
  N Strips:   0
  X Bounds:   2.318e+02, 2.382e+02
  Y Bounds:   3.943e+01, 5.107e+01
  Z Bounds:   -7.000e+01, 0.000e+00
  N Arrays:   2

In [8]:
pl = pv.Plotter()
pl.add_mesh(sub, show_edges=True)
pl.set_scale(zscale=MESH_Z_SCALE)
pl.show()

Widget(value="<iframe src='http://localhost:61908/index.html?ui=P_0x161a23190_1&reconnect=auto' style='width: …

## Save out to `.msh`

In [9]:
pv.save_meshio(OUT_PATH, sub)
OUT_PATH

'2022_11_30_11_52_49_mesh_geometry.msh'

## Comparison of Subdivisions

In [10]:
pl = pv.Plotter(shape=(2, 3))
for i, method in enumerate(['linear', 'butterfly', 'loop']):
    submethod = surface.subdivide(
        nsub=N_SUB,           # Number of subdivisions
        subfilter=method,     # Subdividing algorithm
        progress_bar=True,    # Monitor progress for larger meshes
    )
    # Compute a cell quality metric to make visual comparison easier
    qual = submethod.compute_cell_quality('area')
    
    pl.subplot(0, i)
    pl.add_text(method)
    pl.add_mesh(qual, show_edges=False, show_scalar_bar=False)
    pl.set_scale(zscale=MESH_Z_SCALE)
    
    pl.subplot(1, i)
    pl.add_mesh(submethod, show_edges=True)
    pl.set_scale(zscale=MESH_Z_SCALE)
    
    
    
pl.link_views()
pl.show()

Subdividing Mesh: 100%|██████████[00:00<00:00]
Subdividing Mesh: 100%|██████████[00:00<00:00]
Subdividing Mesh: 100%|██████████[00:00<00:00]


Widget(value="<iframe src='http://localhost:61908/index.html?ui=P_0x17cd28950_2&reconnect=auto' style='width: …